In [1]:
import onnx
import onnx.helper as helper
import numpy as np

In [2]:
onnx_model = onnx.load("test_add.onnx")
graph = onnx_model.graph
nodes = graph.node

In [3]:
def make_test_add_node(inputs_name, output_name,node_name = 'test_add_0', bias_value = 2.):
    test_add_node = helper.make_node(
        op_type = 'TestAdd',
        name = node_name,
        inputs = inputs_name,
        outputs = output_name,
        alpha = bias_value
    )
    return test_add_node

In [4]:
remove_node_names = ['Add_5', 'Add_6','Constant_4']

In [5]:
remove_node_idxs = []


In [6]:
new_node_input_names = ['x_out','y_out']
new_node_output_name = ['onnx::Gemm_12']

In [7]:
 for i in range(len(nodes)):
    node_name = nodes[i].name
    if node_name == 'Gemm_7':
        print(f'{node_name} : {nodes[i]}')
        nodes.insert(
            i,
            make_test_add_node(
                new_node_input_names,
                new_node_output_name,
            )
        )
        break

Gemm_7 : input: "onnx::Gemm_12"
input: "linear2.weight"
input: "linear2.bias"
output: "out"
name: "Gemm_7"
op_type: "Gemm"
attribute {
  name: "alpha"
  f: 1.0
  type: FLOAT
}
attribute {
  name: "beta"
  f: 1.0
  type: FLOAT
}
attribute {
  name: "transB"
  i: 1
  type: INT
}



In [8]:
for i in range(len(nodes)):
    node_name = nodes[i].name
    print(f'id : {i} , node_name :{node_name}')
    if node_name in remove_node_names:
        remove_node_idxs.append(i)
        print(f'{node_name} : {nodes[i]}')

id : 0 , node_name :Gemm_0
id : 1 , node_name :Constant_1
id : 2 , node_name :Add_2
id : 3 , node_name :Gemm_3
id : 4 , node_name :Constant_4
Constant_4 : output: "onnx::Add_10"
name: "Constant_4"
op_type: "Constant"
attribute {
  name: "value"
  t {
    data_type: 1
    raw_data: "\000\000\000@"
  }
  type: TENSOR
}

id : 5 , node_name :Add_5
Add_5 : input: "x_out"
input: "onnx::Add_10"
output: "onnx::Add_11"
name: "Add_5"
op_type: "Add"

id : 6 , node_name :Add_6
Add_6 : input: "onnx::Add_11"
input: "y_out"
output: "onnx::Gemm_12"
name: "Add_6"
op_type: "Add"

id : 7 , node_name :test_add_0
id : 8 , node_name :Gemm_7
id : 9 , node_name :Relu_8


In [9]:
remove_node_idxs.reverse()

In [10]:
for remove_node_idx in remove_node_idxs:
    print(nodes[remove_node_idx])
    nodes.remove(nodes[remove_node_idx])

input: "onnx::Add_11"
input: "y_out"
output: "onnx::Gemm_12"
name: "Add_6"
op_type: "Add"

input: "x_out"
input: "onnx::Add_10"
output: "onnx::Add_11"
name: "Add_5"
op_type: "Add"

output: "onnx::Add_10"
name: "Constant_4"
op_type: "Constant"
attribute {
  name: "value"
  t {
    data_type: 1
    raw_data: "\000\000\000@"
  }
  type: TENSOR
}



In [11]:
onnx.save(onnx_model, 'edited_model.onnx')